# Hierarchical clustering demo
A re-creation of Mike Eisen's demonstration of how hierarchical clustering can find preexisting structure in data. Optimal node ordering is the key.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
%matplotlib inline

Select an image file.

In [ ]:
# filename = "images/man.jpg"    # Excellent (inverted)!
# Mike Eisen's original image!
# filename = "images/Raphael-The-Virgin-and-Child-with-St-John-the-Baptist-1507.jpg"  # Color. Excellent (inverted)
filename = "images/einstein.jpg"  # Excellent!
# filename = "images/fridaKahlo.jpg"  # Excellent (inverted)!

Read in image.

In [ ]:
img = imread(filename)

May show "color" even for B/W image, unless cmap is set to gray!

In [ ]:
plt.imshow(img);

The array shapes vary depending on the image format, so we take the first layer only. It shouldn't matter how the image was read in.

In [ ]:
if len(img.shape) == 3:
    nimg = img[: , :, 0]  # This takes only the "red" channel. Okay for B/W, but inaccurate if orignal was color.
elif len(img.shape) == 2:
    nimg = img

Grab the dimensions and check. These are used later for plotting.

In [ ]:
x, y = nimg.shape
print("({}, {})".format(x, y))

Number of rows is height and number of columns is width, rescale for plotting.

In [ ]:
h, w = 5/(y/x), 5  # Maintain aspect ratio.

In [ ]:
plt.imshow(nimg);

In [ ]:
plt.matshow(nimg, cmap='gray');

Reduce the resolution by taking every other value in both directions.

In [ ]:
nimg = nimg[[i % 2 == 0 for i in range(x)], :]
nimg = nimg[:, [i % 2 == 0 for i in range(y)]]

In [ ]:
plt.matshow(nimg, cmap='gray');

Convert numpy array to dataframe and then use the .sample method to randomly sample rows, but keep fraction = 1 so that *all* rows are returned, scrambling the image by rows.

In [ ]:
scrambled_nimg = pd.DataFrame(nimg).sample(frac=1)

Take a look at the row-scrambled image. matshow works with numpy array and pandas dataframe.

In [ ]:
plt.matshow(scrambled_nimg, cmap = 'gray');

Straight up clustering doesn't quite do it.

In [ ]:
cg = sns.clustermap(scrambled_nimg, col_cluster = False, method = 'median', 
                    metric = 'euclidean', cmap = "gray", figsize=(w,h))
cg.ax_row_dendrogram.set_visible(False)
cg.ax_col_dendrogram.set_visible(False)
cg.cax.set_visible(False);  # Turn off the color scale.

For optimal ordering, see [scipy.cluster.hierarchy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage) and [stackoverflow](https://stackoverflow.com/questions/38705359/how-to-give-sns-clustermap-a-precomputed-distance-matrix). First, make a `linkage` object using optimal leaf ordering, which minimizes the distance between adjacent vectors *even when they belong to different branches*. The linkage is passed to the next step in clustermap.

In [ ]:
Z = linkage(scrambled_nimg, method='single', optimal_ordering=True)

Voilà!

In [ ]:
cg = sns.clustermap(scrambled_nimg, row_linkage=Z, col_cluster = False, figsize=(w,h), cmap='gray')
cg.ax_col_dendrogram.set_visible(False)
cg.cax.set_visible(False);

To test after scrambling rows *and* columns, scramble the columns also in the row-scrambled matrix.

In [ ]:
scrambled_nimg = pd.DataFrame(scrambled_nimg.T).sample(frac=1).T

Row and column scrambled image.

In [ ]:
plt.matshow(scrambled_nimg, cmap = 'gray');

Straight up clustering of rows and columns.

In [ ]:
cg = sns.clustermap(scrambled_nimg, method = 'median', 
                    metric = 'euclidean', cmap = "gray", figsize=(w,h))
cg.ax_row_dendrogram.set_visible(False)
cg.ax_col_dendrogram.set_visible(False)
cg.cax.set_visible(False);  # Turn off the color scale.

Linkage with optimal ordering for the columns. Note the shape is different.

In [ ]:
ZC = linkage(scrambled_nimg.T, method='single', optimal_ordering=True)

In [ ]:
ZC.shape

Behold the magic of clustering!

In [ ]:
cg = sns.clustermap(scrambled_nimg, row_linkage=Z, col_linkage=ZC, figsize=(w,h), cmap='gray')
cg.cax.set_visible(False);